In [46]:
import pandas as pd
import numpy as np

def wraptable(thing, caption):
    # replace ascii bs
    thing = thing.replace("\\textasciicircum \\{a\\}", "\\textsuperscript{a}").replace("\\textasciicircum \\{b\\}", "\\textsuperscript{b}" )
    for substr in ['MUTAG', 'ER\\textsuperscript{a}', 'BA\\textsuperscript{a}', 'HK\\textsuperscript{a}', 'WC\\textsuperscript{a}', 'ForcedRB &']:
        thing = thing.replace(substr, f'\midrule \n{substr}', 1)
    thing = thing.replace("@@@", "")
    thing = thing.replace("Nmin", "$N_\\text{min}$")
    thing = thing.replace("Nmax", "$N_\\text{max}$")
    thing = thing.replace("gurobi", 'Gurobi')
    thing = thing.replace("LiftMP", 'OptGNN')
    replace = {'REDDIT-BINARY': '\midrule \nREDDIT-BIN\\textsuperscript{c}',
           'REDDIT-MULTI-12K': 'REDDIT-M-12K\\textsuperscript{c}',
            'REDDIT-MULTI-5K': 'REDDIT-M-5K\\textsuperscript{c}',
            'gen\\_n': "$N_\\text{min}$",
            'greedy': 'Greedy',
            'Type': 'Dataset',
            'dataset': 'Dataset'} 
    for k, v in replace.items():
        thing = thing.replace(k, v)
    return thing
    #return "\\begin{table} \n" + thing + f"\n\\caption{{ {caption}}} \n" + "\\end{table}\n"


def do_bolding(latex_string, list_of_things):
    dict_of_things = {str(x): f'\\textbf{{{str(x)}}}' for x in list_of_things}
    for k, v in dict_of_things.items():
        latex_string = latex_string.replace(k, v)
    return latex_string
    
    
    

In [47]:
problem = 'max_cut'
#problem = 'vertex_cover'

In [48]:
fname = f'Table1_{problem}.csv'
#fname = 'Table1_VC.csv'
df = pd.read_csv(fname)
df = df[~df.dataset.isin(['RANDOM', 'ForcedRB'])]
dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
                              ['BA^{a}', 'ER^{a}', 'HK^{a}', 'WC^{a}', 'MUTAG^{b}', 'ENZYMES^{b}', 'PROTEINS^{b}',   'IMDB-BIN^{b}', 'COLLAB^{b}',]))

df.dataset = df.dataset.map(lambda x: dataset_conversion.get(x, x))
#df = pd.read_csv('Table1_VC.csv')

In [49]:
gen_n_dict = dict(zip([50, 100, 400, "", np.nan], [100, 200, 500, "", ""]))
nudf = df.round(2).fillna("")
nudf.gen_n = nudf.gen_n.apply(lambda x: int(x) if x != "" else x)
nudf['Nmax'] = nudf.gen_n.apply(lambda x: gen_n_dict[x])
nudf['Nmin'] = nudf.gen_n
nudf.dataset = nudf.apply(lambda row: f'{row.dataset} ({row.Nmin},{row.Nmax})', axis=1)
nudf.dataset = nudf.dataset.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)

nudf = nudf[['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']]

# get things to bold
list_of_things = []
for i, row in nudf.iterrows():
    possibilities = [float(x.split(" ")[0]) for x in row.values[2:] if x!=""]
    if problem == 'max_cut':
        list_of_things.append(f'{max(possibilities):0.2f}')
    else:
        list_of_things.append(f'{min(possibilities):0.2f}')

main_body = (nudf.to_latex(index=False))
print(wraptable(do_bolding(main_body,list_of_things),fname))

\begin{tabular}{llllll}
\toprule
         Dataset &            GAT &           GCNN &            GIN &      GatedGCNN &         OptGNN \\
\midrule
 \midrule 
ER\textsuperscript{a} (50,100) &    525.92 (25) &    500.94 (17) &    498.82 (14) &    526.78 (14) &    \textbf{528.95} (18) \\
ER\textsuperscript{a} (100,200) &   1979.45 (20) &   1890.10 (26) &   1893.23 (23) &   1978.78 (21) &   \textbf{1995.05} (24) \\
ER\textsuperscript{a} (400,500) & 16317.69 (208) & 15692.12 (233) & 15818.42 (212) & 16188.85 (210) & \textbf{16387.46} (225) \\
       \midrule 
MUTAG\textsuperscript{b} &     27.84 (19) &     27.11 (12) &     27.16 (13) &     \textbf{27.95} (14) &      \textbf{27.95} (9) \\
     ENZYMES\textsuperscript{b} &     80.73 (17) &     74.03 (12) &     73.85 (16) &      81.35 (9) &     \textbf{81.37} (14) \\
    PROTEINS\textsuperscript{b} &    100.94 (14) &     92.01 (19) &     92.62 (17) &    101.68 (10) &    \textbf{102.15} (12) \\
    IMDB-BIN\textsuperscript{b} &     81.89 (18) &

/tmp/ipykernel_1821306/1585800806.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = (nudf.to_latex(index=False))


In [50]:
nudf.keys()

Index(['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP'], dtype='object')

# Table 2

In [51]:
fname = f'Table2_{problem}.csv'
#fname = 'Table2_VC.csv'

df = pd.read_csv(fname)
df = df[~df['Unnamed: 0'].isin(['RANDOM', 'ForcedRB'])]

#dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
#                              ['BA^{b}', 'ER^{b}', 'PC^{b}', 'WC^{b}', 'MUTAG^{a}', 'ENZYMES^{a}', 'PROTEINS^{a}',   'IMDB-BINARY^{a}', 'COLLAB^{a}',]))

df.Type = df.Type.map(lambda x: dataset_conversion.get(x, x))#df = pd.read_csv('Table2_VC.csv')


df

# load in nikos baselines
if fname == 'Table2_VC.csv':
    x = pd.read_csv('kamis_lwd_baselines.csv')
    problems = {'er': 'ErdosRenyi_400.0',
            'ba': 'BarabasiAlbert_400.0',
            'hk': 'PowerlawCluster_400.0',
            'ws': 'WattsStrogatz_400.0',}

    for i, row in x.iterrows():
        problem = problems[row['problem']]
        df.loc[df['Unnamed: 0'] == problem, 'kamis'] = f'{row["k_solution_sizes_average"]:0.2f} ({row["k_times_average"]:0.2f})'
        df.loc[df['Unnamed: 0'] == problem, 'lwd'] = f'{row["l_solution_sizes_average"]:0.2f} ({row["l_times_average"]:0.2f})'
        
    

In [52]:
df

,Unnamed: 0,Type,Nmin,Nmax,LiftMP,greedy,gurobi_0.1,gurobi_0.5,gurobi_1.0,gurobi_2.0,gurobi_4.0,gurobi_8.0
0,BarabasiAlbert_50.0,BA^{a},50.0,100.0,351.49 (18),200.10,351.87,352.10,352.12,352.12,352.12,352.12
1,BarabasiAlbert_100.0,BA^{a},100.0,200.0,717.19 (20),407.98,719.41,719.51,719.72,719.80,719.89,720.17
2,BarabasiAlbert_400.0,BA^{a},400.0,500.0,2197.99 (66),1255.22,2208.11,2208.11,2208.11,2208.62,2212.42,2212.49
3,ErdosRenyi_50.0,ER^{a},50.0,100.0,528.95 (18),298.55,529.93,529.98,530.03,530.08,530.11,530.16
4,ErdosRenyi_100.0,ER^{a},100.0,200.0,1995.05 (24),1097.26,2002.88,2002.88,2002.88,2002.88,2002.90,2002.93
5,ErdosRenyi_400.0,ER^{a},400.0,500.0,16387.46 (225),8622.34,16476.72,16481.57,16491.60,16494.35,16495.06,16495.31
6,PowerlawCluster_50.0,HK^{a},50.0,100.0,345.74 (18),196.23,346.18,346.42,346.42,346.42,346.42,346.42
7,PowerlawCluster_100.0,HK^{a},100.0,200.0,709.39 (23),402.54,711.68,711.88,712.26,712.46,712.62,712.88
8,PowerlawCluster_400.0,HK^{a},400.0,500.0,2159.90 (61),1230.98,2169.46,2169.46,2169.46,2169.86,2173.76,2173.88
9,WattsStrogatz_50.0,WC^{a},50.0,100.0,198.29 (18),116.65,198.74,198.74,198.74,198.74,198.74,198.74


In [53]:
nudf = df.iloc[:,1:].round(2).fillna("")
#
#print(nudf.keys())



nudf.Nmin = nudf.Nmin.apply(lambda x: int(x) if x!="" else x)
nudf.Nmax = nudf.Nmax.apply(lambda x: int(x) if x!="" else x)
nudf.Type = nudf.apply(lambda row: f'{row.Type} ({row.Nmin},{row.Nmax})', axis=1)
nudf.Type = nudf.Type.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)
#nudf['dataset'] = nudf.Type
nudf = nudf[[x for x in nudf.keys() if x not in ['Nmin', 'Nmax', 'SDP proj'
]]]

# TODO: get things to bold
# get things to bold

list_of_things = []
for i, row in nudf.iterrows():
    #print(row)
    possibilities = [float(x.split(" ")[0]) if isinstance(x, str) else x for x in row.values[2:]]
    if problem == 'max_cut':
        list_of_things.append(f'{max(possibilities):0.2f}')
    else:
        list_of_things.append(f'{min(possibilities):0.2f}')
list_of_things = []

nudf = nudf[['Type', 'LiftMP', 'greedy', 'gurobi_0.1', 'gurobi_1.0', 'gurobi_8.0']]


cols = nudf.columns
hrow1 = []
hrow2 = []
for x in cols:
    if x.startswith('gurobi'):
        hrow1.append("gurobi")
        hrow2.append(x.split("_")[1] + "s")
    else:
        hrow1.append(x)
        hrow2.append("@@@")

header = pd.MultiIndex.from_arrays([hrow1, hrow2])
nudf.columns = header

main_body = nudf.to_latex(index=False)

print(wraptable(do_bolding(main_body,list_of_things),fname))



\begin{tabular}{llrrrr}
\toprule
            Dataset &         OptGNN &  Greedy & \multicolumn{3}{l}{Gurobi} \\
              &             &      &     0.1s &     1.0s &     8.0s \\
\midrule
 \midrule 
BA\textsuperscript{a} (50,100) &    351.49 (18) &  200.10 &   351.87 &   352.12 &   352.12 \\
BA\textsuperscript{a} (100,200) &    717.19 (20) &  407.98 &   719.41 &   719.72 &   720.17 \\
BA\textsuperscript{a} (400,500) &   2197.99 (66) & 1255.22 &  2208.11 &  2208.11 &  2212.49 \\
 \midrule 
ER\textsuperscript{a} (50,100) &    528.95 (18) &  298.55 &   529.93 &   530.03 &   530.16 \\
ER\textsuperscript{a} (100,200) &   1995.05 (24) & 1097.26 &  2002.88 &  2002.88 &  2002.93 \\
ER\textsuperscript{a} (400,500) & 16387.46 (225) & 8622.34 & 16476.72 & 16491.60 & 16495.31 \\
 \midrule 
HK\textsuperscript{a} (50,100) &    345.74 (18) &  196.23 &   346.18 &   346.42 &   346.42 \\
HK\textsuperscript{a} (100,200) &    709.39 (23) &  402.54 &   711.68 &   712.26 &   712.88 \\
HK\textsuperscript

/tmp/ipykernel_1821306/2403306521.py:45: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = nudf.to_latex(index=False)


# instructions: 
2. add midrules